In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def parse_nab_file(fpath):
    """
    In NAB export, net_price is total after fees,
    and ppu is before fees.
    """
    df = pd.read_csv(fpath, header=0, names=[
        'date', 'conf', 'ticker', 'qty', 'type', 'ppu', 'fees', 'net_price', 'note'
    ])
    df['type'] = df['type'].str.lower()
    df['qty'] = pd.to_numeric(df['qty']).astype(int)
    df['ppu'] = df['net_price'] / df['qty']  # We need net unit cost base
    df['ppu'] = pd.to_numeric(df['ppu']).astype(float)
    df['date'] = pd.to_datetime(df['date'])
    return df[['date', 'type', 'ticker', 'qty', 'ppu']]

In [ ]:
def sort_by_date(trades: pd.DataFrame) -> pd.DataFrame:
    """
    Sort trades chronologically, oldest at top
    """
    return trades.sort_values(['date'], ascending=True)


def identify_sold_stocks(trades: pd.DataFrame) -> pd.Series:
    """
    Find all unique tickers that have been sold
    """
    sells = trades[trades['type']=='sell']
    tickers = sells.drop_duplicates('ticker')['ticker']
    return tickers


def explode_trades(trades: pd.DataFrame):
    def explode_one(r: pd.Series) -> pd.DataFrame:
        """
        Returns a dataframe reflecting the exploded trade
        """
        n = r['qty']
        r['qty'] = 1
        return pd.DataFrame(
            np.repeat(pd.DataFrame([r]).values, n, axis=0),
            columns=r.index
        )
    
    def explode_many(trades: pd.DataFrame) -> pd.DataFrame:
        """
        Returns a df reflecting the exploded input df trades
        """
        many = trades.apply(explode_one, axis=1)
        return pd.concat(list(many), ignore_index=True)
    return explode_many(trades)


def split_buysells(trades: pd.DataFrame) -> dict:
    """
    Separate buys and sells into separate dfs
    """
    return {
        'buys': trades[trades['type']=='buy'],
        'sells': trades[trades['type']=='sell']
    }

def amputate_unused_buys(buysells: dict) -> dict:
    """
    Only the stocks which have now been disposed are relevant for CGT.
    Unsold stocks are irrelevant.
    """
    n_buys = len(buysells.buys)
    n_sells = len(buysells.sells)
    if n_sells > n_buys:
        raise Exception("This is impossible! Are we missing data?")
    return {
        'buys': buysells.buys.iloc[0: n_sells - 1],
        'sells': buysells.sells
    }


def enrich_sales(buysells: dict) -> dict:
    """
    Takes the exploded buys and sells for one specific stock,
    and appends calculation results for each sale
    """
    buys = buysells['buys'].copy().reset_index()
    sells = buysells['sells'].copy().reset_index()
    sells['gain_loss'] = sells['ppu'] - buys['ppu']
    sells['days_held'] = sells['date'] - buys['date']
    sells['12month_discount'] = sells['days_held'].dt.days > 365
    sells['gainloss_after_discount'] = np.where(
        sells['12month_discount'],
        sells['gain_loss'] / 2,
        sells['gain_loss']
    )
    sells['qty'] = sells['qty'].astype(float)
    sells['ppu'] = sells['ppu'].astype(float)
    sells['gain_loss'] = sells['gain_loss'].astype(float)
    sells['gainloss_after_discount'] = sells['gainloss_after_discount'].astype(float)
    return {
        'buys': buys,
        'sells': sells
    }


def summary_report(all_exploded: pd.DataFrame) -> pd.DataFrame:
    """"
    Generate a auman-readable summary of exploded sales
    """
    report = all_exploded.pivot_table(index=['12month_discount', 'date', 'ticker'],
               margins=True,
               margins_name='total',  # defaults to 'All'
               aggfunc=sum)
    
    # Rejig headers for clarity
    report.columns = ['gain_loss', 'gain_after_discount', 'total_sale_price', 'volume']
    return report[['volume', 'total_sale_price', 'gain_loss', 'gain_after_discount']]


def main() -> pd.DataFrame:
    trades = parse_nab_file('confirmed_trades.csv')
    tickers = identify_sold_stocks(trades)
    exploded = explode_trades(trades)
    exploded = sort_by_date(exploded)
    out = []
    for t in tickers:
        subset = exploded[exploded['ticker']==t]
        buysells = split_buysells(subset)
        buysells = enrich_sales(buysells)
        out.append(buysells['sells'])
    
    out = pd.concat(out, ignore_index=True).drop(columns='index')
    report = summary_report(out)
    return report
    

In [ ]:
"""
Requires the NAB "trades" export in current dir,
with name `confirmed_trades.csv`
"""
main()